### verl: vllm & fsdp

- torch
    - 调用 `empty_cache()` 会释放 PyTorch 中所有未使用的缓存内存，以便其他 GPU 应用程序可以使用这些内存。
- vllm
    - vllm has its own caching memory allocator CuMemAllocator
        - https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/device_allocator/cumem.py#L103
- https://github.com/vllm-project/vllm/issues/15254
    - (old_weights + kv_cache) + new_weights
```python
params = self.fsdp_module.state_dict() # new weights on GPU
self.inference_engine.wake_up() # old weights + kv_cache memory allocated -> this can cause OOM for big models!
self.inference_engine.llm_engine.model_executor.driver_worker.worker.model_runner.model.load_weights([(name, param) for name, param in params.items())])
```